#ML Assignment Fine Tune a Model

# Installing Libraries and Dependencies

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.9 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import T5Tokenizer,T5ForConditionalGeneration,Adafactor

In [ ]:
t5_tokenizer = T5Tokenizer.from_pretrained("t5-base")


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
print(t5_tokenizer)

T5Tokenizer(name_or_path='t5-base', vocab_size=32100, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_44

In [ ]:
t5_model = T5ForConditionalGeneration.from_pretrained("t5-base")

# Retrieving the data from the given PDFs.

In [ ]:
titles = []
articles = []

pdf_directory = "/content/sample_data/research"

In [ ]:
!pip install PyPDF2
import os
from PyPDF2 import PdfReader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.6 MB/s eta 0:00:00


In [ ]:
article_data = []

# Iterate through the files in the folder
# Iterate through the files in the folder
# Iterate over each file in the directory
for filename in os.listdir(pdf_directory):
    # Check if the file is a PDF
    if filename.endswith('.pdf'):
        # Construct the full path of the PDF file
        pdf_path = os.path.join(pdf_directory, filename)

        # Open the PDF file and create a PdfReader object
        with open(pdf_path, 'rb') as pdf_file:
            pdf_reader = PdfReader(pdf_file)
            article_name = os.path.splitext(filename)[0]
            # Initialize an empty string to store the extracted text
            extracted_text = ""

            # Iterate through each page of the PDF
            for page in pdf_reader.pages:
                # Extract text from the page
                text = page.extract_text()
                # Append the extracted text to the overall text
                extracted_text += text
            
            articles.append(extracted_text)
            titles.append(article_name)


In [ ]:
titles

['Human Papillomavirus Vaccination for Adults Updated Recommendations of the Advisory Committee on Immunization Practices',
 'Use of a 2-Dose Schedule for Human Papillomavirus Vaccination — Updated Recommendations of the Advisory Committee on Immunization Practices',
 'Employment and Activity Limitations Among Adults with Chronic Obstructive Pulmonary Disease',
 'Recommendations of the Advisory Committee on Immunization Practices']

In [ ]:
articles[1]

'Morbidity and Mortality Weekly ReportMMWR  / December 16, 2016  / V ol. 65  /  No. 49  1405\nUS Department of Health and Human Services/Centers for Disease Control and PreventionIntroduction\nVaccination against human papillomavirus (HPV) is rec -\nommended to prevent HPV infections and HPV-associated \ndiseases, including cancers. Routine vaccination at age 11 or \n12 years has been recommended by the Advisory Committee \non Immunization Practices (ACIP) since 2006 for females and since 2011 for males ( 1,2). This report provides recom -\nmendations and guidance regarding use of HPV vaccines and updates ACIP HPV vaccination recommendations previously published in 2014 and 2015 ( 1,2). This report includes new \nrecommendations for use of a 2-dose schedule for girls and boys who initiate the vaccination series at ages 9 through 14 years. Three doses remain recommended for persons who initiate the vaccination series at ages 15 through 26 years and for immunocompromised persons.\nBackgr


# Preprocessing 
---


**Tokenizes the passages, converts them to lowercase, removes punctuation, and eliminates common stop words to prepare the text for further analysis or processing.**


---



In [ ]:
import nltk
import string
from nltk.corpus import stopwords

nltk.download('stopwords')

passage_titles = []
passages = []

# Define a set of punctuation characters
punctuation = set(string.punctuation)

# Get the set of stop words
stop_words = set(stopwords.words('english'))

for i in range(len(titles)):
    title = titles[i]
    article = articles[i]

    if len(article) == 0:
        print('Skipping empty article:', title)
        continue

    article = article.replace('\n', '')  # Remove newline characters from the article

    # Preprocess the passage
    chunk_size = 700  # Number of characters in each chunk
    start = 0
    end = start + chunk_size

    while start < len(article):
        chunk = article[start:end]

        # Lowercase the chunk
        chunk = chunk.lower()

        # Remove punctuation from the chunk
        chunk = ''.join(char for char in chunk if char not in punctuation)

        # Remove stop words from the chunk
        chunk = ' '.join(word for word in chunk.split() if word not in stop_words)

        if len(chunk) > 0:
            passage_titles.append(title)
            passages.append(chunk)

        start = end
        end = start + chunk_size


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
len(passages)

464

In [ ]:
passages[1]

'1 12 years vaccination given starting age 9 years catchup vaccination recommended since 2006 females age 26 years since 2011 males age 21 years certain special populations age 26 years report updates acip catchup hpv vaccination recommendations guidance published 2014 2015 2016 1–3 routine recommendations vaccination adolescents changed june 2019 acip recommended catchup hpv vaccination persons age 26 years acip recommend catchup vaccination adults aged 27 45 years recognized persons adequately vaccinated might b'

# DPR Retriever



---


**DPR Context Encoder:** The code initializes an instance of the DPRContextEncoder from the "facebook/dpr-ctx_encoder-multiset-base" pretrained model. This DPR context encoder is used to encode and represent passages of text.

**DPR Context Encoder Tokenizer:** The code also initializes a tokenizer, DPRContextEncoderTokenizerFast, associated with the DPR context encoder model. And this is used to tokenize the passages for DPR Context Encoder



---


In [ ]:
from transformers import DPRQuestionEncoder, DPRContextEncoder, DPRQuestionEncoderTokenizerFast, DPRContextEncoderTokenizerFast
import torch

In [ ]:
# Load the DPR context encoder and tokenizer
ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-multiset-base")
ctx_tokenizer = DPRContextEncoderTokenizerFast.from_pretrained("facebook/dpr-ctx_encoder-multiset-base")

# Set device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to device
ctx_encoder = ctx_encoder.to(device=device)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity

# def is_relevant(question_embedding, passage):
#     # question_embedding = embed_question(question)
#     passage_embedding = embed_passage(passage)
    
#     similarity_score = cosine_similarity(question_embedding, passage_embedding)
#     print(f"Similarity score: {similarity_score}")
#     # Define a threshold to determine relevance
#     relevance_threshold = 0.525
    
#     if similarity_score > relevance_threshold:
#         return True
#     else:
#         return False

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
!pip install -U scikit-learn
!pip install rank-bm25

from sklearn.metrics.pairwise import cosine_similarity


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi



---


1. **Tokenization and Preprocessing:** The function takes a query and a list of passages as input. It tokenizes both the query and passages using the TfidfVectorizer's tokenization function. 

2. **BM25 Scores Calculation:** The code calculates BM25 scores for the passages based on their relevance to the query. 

    BM25 is a ranking function commonly used in information retrieval. The BM25Okapi class is initialized with the passage tokens, and the get_scores() method is called with the query tokens to obtain the BM25 scores. 


3. **Passage Classification:** The code classifies the passages as positive or negative based on a relevance threshold (relevance_threshold). Passages with scores higher than the threshold are considered positive, while passages with scores lower or equal to the threshold are considered negative. 


---



In [ ]:
def classify_passages(query, passages, relevance_threshold=0.5):
    # Tokenize and preprocess the query and passages
    tokenizer = TfidfVectorizer().build_tokenizer()
    query_tokens = tokenizer(query)
    passage_tokens = [tokenizer(passage) for passage in passages]

    # Calculate the BM25 scores
    bm25 = BM25Okapi(passage_tokens)
    scores = bm25.get_scores(query_tokens)
     # Print the scores
    print("Scores:")
    for score in scores:
        print("{:.6f}".format(score))
    # Classify passages as positive or negative based on relevance threshold
    positive_passages = []
    negative_passages = []
    for passage, score in zip(passages, scores):
        if score > relevance_threshold:
            positive_passages.append(passage)
        else:
            negative_passages.append(passage)

    return positive_passages, negative_passages


In [ ]:
def embed_question(question):
    q_encoder_model_name = "facebook/dpr-question_encoder-single-nq-base"
    q_tokenizer = DPRQuestionEncoderTokenizerFast.from_pretrained(q_encoder_model_name)
    q_encoder = DPRQuestionEncoder.from_pretrained(q_encoder_model_name)
    
    input_ids = q_tokenizer.encode(question, truncation=True, padding="longest", return_tensors="pt")
    input_ids = input_ids.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    
    with torch.no_grad():
        outputs = q_encoder(input_ids=input_ids)
    
    embeddings = outputs.pooler_output
    
    return embeddings

def embed_passage(passage):
    ctx_encoder_model_name = "facebook/dpr-ctx_encoder-single-nq-base"
    ctx_tokenizer = DPRContextEncoderTokenizerFast.from_pretrained(ctx_encoder_model_name)
    ctx_encoder = DPRContextEncoder.from_pretrained(ctx_encoder_model_name)
    
    input_ids = ctx_tokenizer.encode(passage, truncation=True, padding="longest", return_tensors="pt")
    input_ids = input_ids.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    
    with torch.no_grad():
        outputs = ctx_encoder(input_ids=input_ids)
    
    embeddings = outputs.pooler_output
    
    return embeddings 


In [ ]:

# positive_passages = []
# negative_passages = []
question = "When did the GARDASIL 9 recommendations change? What were the past three recommendation changes for GARDASIL 9? Is GARDASIL 9 recommended for Adults? Does the ACIP recommend one dose GARDASIL 9?"
# question_embedding = embed_question(question)
# # for passage in passages:
# #   if is_relevant(question_embedding, passage) == True:
# #     positive_passages.append(passage)
# #   else :
# #     negative_passages.append(passage)

positive_passages, negative_passages = classify_passages(question, passages)
print(f"Positive Passages:{len(positive_passages)}")

print(f"\nNegative Passages:{len(negative_passages)}")


Scores:
1.597676
10.256800
2.248296
2.726159
1.608010
2.832543
8.269542
0.000000
1.695754
2.233744
4.950032
0.000000
4.949462
0.000000
2.092012
4.729205
1.650717
0.000000
3.544630
5.576767
2.248296
0.000000
0.000000
3.822865
5.576767
1.695754
0.000000
0.000000
2.306269
1.618478
0.000000
0.000000
0.000000
0.000000
0.000000
3.749634
8.021948
4.252947
3.749634
1.661750
2.349866
2.726159
0.000000
1.577403
3.478969
0.000000
0.000000
0.000000
0.000000
6.160499
8.905530
6.526248
5.880108
0.000000
4.032899
0.000000
4.106091
3.679241
0.000000
0.000000
0.000000
1.528900
1.538360
1.672932
2.253996
7.072148
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
2.317016
3.243882
0.000000
0.000000
0.000000
22.446793
0.000000
0.000000
3.954943
2.144441
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
5.083464
0.000000
0.000000
0.000000
0.000000
4.839860
0.000000
3.094107
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
4.249670
0.000000
0.000000
0.000000
0.000000
0.000000




---


* **Here, a dataset that pairs positive passages with randomly selected subsets of negative passages is generated.**

* **Each example in the dataset consists of a positive passage and three associated negative passages. And this is used for passage ranking and information retrieval.**



---



In [ ]:
import random
dataset = []

for positive_passage in positive_passages:
    # Randomly select a subset of negative passages
    negative_subset = random.sample(negative_passages, k=3)
    
    # Create a dictionary with positive and negative passages
    example = {
        "positive_passages": positive_passage,
        "negative_passages": negative_subset
    }
    
    # Add the example to the dataset
    dataset.append(example) 

#Contrastive Learning



---


1. **Positive and Negative Examples:** The 
code pairs positive passages (retrieved based on relevance to a query) with randomly selected negative passages. Positive passages are considered relevant to the query, while negative passages are considered irrelevant.

2. **Siamese-like Architecture:** The code utilizes a siamese-like architecture, where both the positive and negative passages are encoded using the same context encoder. This ensures that the embeddings are derived from the same feature space.

3. **Embedding Comparison:** After encoding the positive and negative passages, their embeddings are extracted. The embeddings capture the contextual representation of the passages.

4. **Similarity Calculation: **The code calculates the similarity scores between the positive and negative embeddings using cosine similarity. Higher similarity scores indicate greater similarity between the positive and negative passages.

5. **Margin Ranking Loss:** The contrastive learning objective is to maximize the similarity between positive examples and minimize the similarity between negative examples. The code employs the margin ranking loss function, which encourages a margin (specified by the margin_loss hyperparameter) between the positive and negative similarity scores. The loss penalizes dissimilarity between positive examples and similarity between negative examples.

6. **Optimization:** The loss is backpropagated, and the optimizer updates the parameters of the context encoder based on the computed gradients. The learning rate scheduler adjusts the learning rate over epochs to fine-tune the training process. 



---



In [ ]:
# Define hyperparameters
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
margin_loss = 0.2

# Define optimizer and learning rate scheduler
optimizer = torch.optim.AdamW(ctx_encoder.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

# Define loss function
loss_fn = torch.nn.MarginRankingLoss(margin=margin_loss)

# Indexing and encoding steps

# Define your dataset and other necessary preprocessing steps

# Training loop
num_epochs = 5
margin = 0.3

for epoch in range(num_epochs):
    print(f"Epochs: {epoch}")
    total_loss = 0.0
    ctx_encoder.train()
    for i in range(0, len(dataset)):
        optimizer.zero_grad()

        # Get a batch of positive and negative passages
        batch = dataset[i]
        positive_passages = batch["positive_passages"]
        negative_passages = batch["negative_passages"]

        # Encode passages
      
        positive_encodings = ctx_tokenizer(positive_passages, truncation=True, padding=True, return_tensors="pt").to(device)
        negative_encodings = ctx_tokenizer(negative_passages, truncation=True, padding=True, return_tensors="pt").to(device)

        # Pass the encodings through the context encoder
        positive_outputs = ctx_encoder(**positive_encodings)
        negative_outputs = ctx_encoder(**negative_encodings) 

        # Get the pooler output
        positive_embeddings = positive_outputs.pooler_output
        negative_embeddings = negative_outputs.pooler_output

        # Calculate similarity scores
        similarity_scores = torch.cosine_similarity(positive_embeddings, negative_embeddings)
        # print(f"Simliarity score: {similarity_scores}")
        # similarity_scores.requires_grad = True 

        target = torch.ones_like(similarity_scores)
        target[0] = -1

        # Calculate loss
        # loss = torch.max(torch.zeros_like(target), similarity_scores - similarity_scores[0] + margin).mean()
        # loss = torch.max(torch.zeros_like(target), -similarity_scores + margin).mean()
        # print(f"Loss : {loss}")
        # loss = loss_fn(similarity_scores, target)
        loss = loss_fn(similarity_scores, torch.zeros_like(target), target)
        print(f"Loss : {loss}")
        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    scheduler.step()

    avg_loss = total_loss / len(dataset)
    print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {avg_loss:.4f}")

# Save the trained model
output_dir = "path_to_save_model"
ctx_encoder.save_pretrained(output_dir)

Epochs: 0
Loss : 0.32873761653900146
Loss : 0.29126638174057007
Loss : 0.29248708486557007
Loss : 0.23882804811000824
Loss : 0.27242520451545715
Loss : 0.26231077313423157
Loss : 0.1850196123123169
Loss : 0.21662765741348267
Loss : 0.16705918312072754
Loss : 0.18486924469470978
Loss : 0.1537376046180725
Loss : 0.14537380635738373
Loss : 0.14448942244052887
Loss : 0.1460171937942505
Loss : 0.12436554580926895
Loss : 0.18259753286838531
Loss : 0.16982503235340118
Loss : 0.18792665004730225
Loss : 0.14476056396961212
Loss : 0.1449338048696518
Loss : 0.15690357983112335
Loss : 0.2046574205160141
Loss : 0.14760838449001312
Loss : 0.20242531597614288
Loss : 0.20421560108661652
Loss : 0.13987012207508087
Loss : 0.11741024255752563
Loss : 0.1686539202928543
Loss : 0.16138826310634613
Loss : 0.2262677699327469
Loss : 0.17341725528240204
Loss : 0.10813071578741074
Loss : 0.13260366022586823
Loss : 0.1593995839357376
Loss : 0.3270825743675232
Loss : 0.218398317694664
Loss : 0.2860676348209381
Los

In [ ]:
len(passages)

464

In [ ]:
len(passage_titles)

464

In [ ]:
chunked_corpus = {'title': passage_titles, 'text': passages}

# DPR context encoder to compute embeddings for the passages in batches, and stores the computed embeddings

In [ ]:
from transformers import DPRContextEncoderTokenizerFast
ctx_tokenizer = DPRContextEncoderTokenizerFast.from_pretrained("facebook/dpr-ctx_encoder-multiset-base")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


In [ ]:
num_passages = len(chunked_corpus['title'])

In [ ]:
outputs = ctx_tokenizer(
    chunked_corpus["title"],
    chunked_corpus["text"],
    truncation = True,
    padding = "longest",
    return_tensors = "pt",
)

In [ ]:
input_ids = outputs["input_ids"]

In [ ]:
print(input_ids.shape)

torch.Size([464, 258])


In [ ]:
import torch
device = "cpu"
if torch.cuda.is_available():
  device = torch.device("cuda")

else:
  print("No device is available")

No device is available


In [ ]:
# from transformers import DPRContextEncoder
# ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-multiset-base")
# ctx_encoder = ctx_encoder.to(device=device)

In [ ]:
import time
import datetime

def format_time(elapsed):
  elapsed_rounded = int(round(elapsed))
  return str(datetime.timedelta(seconds = elapsed_rounded))

In [ ]:
import time
import math
torch.set_grad_enabled(False)
t0 = time.time()

step = 0

batch_size = 16

num_passages = input_ids.size()[0]
num_batches =  math.ceil(num_passages/batch_size)

embeds_batches= []
for i in range(0, num_passages, batch_size):
  if step%100 == 0 and not step ==0:
    elapsed = format_time(time.time()-t0)

    print('Batch {:>5,} of {:>5,}. Elapsed: {:}.'. format(step, num_batches, elapsed))

  batch_ids = input_ids[i:i+16, :]
  batch_ids = batch_ids.to(device)

  outputs = ctx_encoder(
      batch_ids,
      return_dict = True,
  )

  embeddings = outputs["pooler_output"]
  embeddings = embeddings.detach().cpu().numpy()

  embeds_batches.append(embeddings)
  step+= 1
  print(step)

print("Done")


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
Done


In [ ]:
num_passages

464

In [ ]:
import numpy as np
embeddings = np.concatenate(embeds_batches, axis = 0)
print('Size of dataset embeddings :', embeddings.shape)

Size of dataset embeddings : (464, 768)


# FAISS

In [ ]:
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 59.6 MB/s eta 0:00:00




---

**The code initializes a HNSWFlat index from faiss for efficient similarity search. It specifies the dimensionality of the embeddings (dim) and the number of connections per index (m). The index is set to use the inner product as the distance metric.**

---



In [ ]:
import faiss
dim = 768
m = 128

index = faiss.IndexHNSWFlat(dim, m , faiss.METRIC_INNER_PRODUCT)

In [ ]:
# t0 = time.time()
index.train(embeddings)
index.add(embeddings)

# print('Done')
# print('Adding embeddings to index took', format_time(time.time()-t0))



---

**Query Encoder and Query Embeddings**

---



In [ ]:
from transformers import DPRQuestionEncoder
q_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-ctx_encoder-multiset-base")
q_encoder = q_encoder.to(device=device)

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-multiset-base were not used when initializing DPRQuestionEncoder: ['ctx_encoder.bert_model.encoder.layer.8.output.dense.bias', 'ctx_encoder.bert_model.encoder.layer.7.attention.self.value.weight', 'ctx_encoder.bert_model.encoder.layer.1.attention.self.value.weight', 'ctx_encoder.bert_model.encoder.layer.0.attention.self.value.bias', 'ctx_encoder.bert_model.encoder.layer.1.output.LayerNorm.bias', 'ctx_encoder.bert_model.encoder.layer.7.attention.self.query.bias', 'ctx_encoder.bert_model.encoder.layer.7.output.dense.bias', 'ctx_encoder.bert_model.encoder.layer.5.attention.self.query.weight', 'ctx_encoder.bert_model.encoder.layer.2.output.dense.weight', 'ctx_encoder.bert_model.encoder.layer.7.output.LayerNorm.weight', 'ctx_encoder.bert_model.encoder.layer.10.attention.output.LayerNorm.bias', 'ctx_encoder.bert_model.encoder.layer.0.attention.self.value.weight', 'ctx_encoder.bert_model.encoder.layer.8.attention.output.LayerNor

In [ ]:
from transformers import DPRQuestionEncoderTokenizerFast

q_tokenizer = DPRQuestionEncoderTokenizerFast.from_pretrained("facebook/dpr-ctx_encoder-multiset-base")
# q_encoder = q_encoder.to(device)

# "Does the ACIP recommend one dose GARDASIL 9?"

In [ ]:
import re

question = "Does the ACIP recommend one dose GARDASIL 9?"

In [ ]:
input_ids = q_tokenizer.encode(question, return_tensors = "pt")
input_ids = input_ids.to(device)
outputs = q_encoder(input_ids)
q_embed = outputs['pooler_output']

q_embed = q_embed.cpu().numpy()

print("Query Embedding:", q_embed.shape)

Query Embedding: (1, 768)




---

**Getting the relevant Passage Indices**

---



In [ ]:
D, I = index.search(q_embed, k= 10)
print("Closest matching indices:", I)

print("Inner Products:", D)

Closest matching indices: [[437 449  72  99 273  93 455 126 110 106]]
Inner Products: [[32.510246 24.621412 24.467623 23.550383 20.609459 20.543941 20.35867
  19.770508 16.162598 15.489602]]




---

**Relevant Passages for the question**

---



In [ ]:
import textwrap 
retrieved_passages = []
retrieved_indices = []
wrapper = textwrap.TextWrapper(width = 80)
for i in I[0]:
  print('Index:', i)
  title = chunked_corpus['title'][i]
  passage = chunked_corpus['text'][i]

  retrieved_passages.append(passage)
  retrieved_indices.append(i)

  print('Article Title:' , title, '\n')
  print('Passage:')

  print(wrapper.fill(passage))
  print('')


Index: 437
Article Title: Recommendations of the Advisory Committee on Immunization Practices 

Passage:
skinner sr et al efficacy human papillomavirus hpv1618 as04adjuvanted vaccine
women aged 15–25 years without serological evidence previous exposure hpv1618
int j cancer 2012131106–16 164 herrero r quint w hildesheim et al reduced
prevalence oral human papillomavirus hpv 4 years bivalent hpv vaccination
randomized clinical trial costa rica plos one 20138e68329 165 kreimer ar
gonzalez p katki ha et al efficacy bivalent hpv 1618 vaccine anal hpv 1618
infection among young women nested analysis within costa rica vaccine rial
lancet oncol 201112862–70 166 naud ps rotelimartins cm de carval

Index: 449
Article Title: Recommendations of the Advisory Committee on Immunization Practices 

Passage:
int j cancer 2013132198–207 207 markowitz le hariri lin c et al reduction human
papillomavirus hpv prevalence among young women following hpv vaccine
introduction united states national health nutr

In [ ]:
retrieved_indices

[437, 449, 72, 99, 273, 93, 455, 126, 110, 106]

In [ ]:
len(passages)

464

In [ ]:
# Save the trained model
output_dir = "path_to_save_model"
ctx_encoder.save_pretrained(output_dir)

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 14.0 MB/s eta 0:00:00


In [ ]:
len(embeddings)

464

In [ ]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import Dataset
import pandas as pd

df = pd.DataFrame(chunked_corpus)
custom_dataset = Dataset.from_pandas(df)
print(custom_dataset)

Dataset({
    features: ['title', 'text'],
    num_rows: 464
})


In [ ]:
embs = []
for i in range(embeddings.shape[0]):
  embs.append(embeddings[i,:])

In [ ]:
custom_dataset = custom_dataset.add_column("embeddings", embs)
custom_dataset

Dataset({
    features: ['title', 'text', 'embeddings'],
    num_rows: 464
})

In [ ]:
index = faiss.IndexHNSWFlat(dim, m, faiss.METRIC_INNER_PRODUCT)
custom_dataset.add_faiss_index(column = "embeddings", index_name = "embeddings", custom_index=index)

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['title', 'text', 'embeddings'],
    num_rows: 464
})

In [ ]:
from transformers import RagRetriever

In [ ]:
# !pip install --upgrade datasets

!pip install transformers
!pip install datasets
!pip install faiss
retriever = RagRetriever.from_pretrained(
    "facebook/rag-sequence-nq",
    use_dummy_dataset = False,
    indexed_dataset = custom_dataset,
    index_name = "embeddings",
)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss


ImportError: ignored

In [ ]:
from transformers import RagTokenizer
tokenizer = RagTokenizer.from_pretrained(
    "facebook/rag-sequence-nq"
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

In [ ]:
from transformers import RagSequenceForGeneration
model = RagSequenceForGeneration.from_pretrained(
    "facebook/rag-sequence-nq",
    retriever = retriever
)

Some weights of the model checkpoint at facebook/rag-sequence-nq were not used when initializing RagSequenceForGeneration: ['rag.question_encoder.question_encoder.bert_model.pooler.dense.weight', 'rag.question_encoder.question_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing RagSequenceForGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RagSequenceForGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RagSequenceForGeneration were not initialized from the model checkpoint at facebook/rag-sequence-nq and are newly initialized: ['rag.generator.lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to us

In [ ]:
context = ""
len = 4
for i in range(4):
  context = context + retrieved_passages[i]


In [ ]:
context

'skinner sr et al efficacy human papillomavirus hpv1618 as04adjuvanted vaccine women aged 15–25 years without serological evidence previous exposure hpv1618 int j cancer 2012131106–16 164 herrero r quint w hildesheim et al reduced prevalence oral human papillomavirus hpv 4 years bivalent hpv vaccination randomized clinical trial costa rica plos one 20138e68329 165 kreimer ar gonzalez p katki ha et al efficacy bivalent hpv 1618 vaccine anal hpv 1618 infection among young women nested analysis within costa rica vaccine rial lancet oncol 201112862–70 166 naud ps rotelimartins cm de carvalint j cancer 2013132198–207 207 markowitz le hariri lin c et al reduction human papillomavirus hpv prevalence among young women following hpv vaccine introduction united states national health nutrition examination surveys 2003–2010 j infect dis 2013208385–93 208 flagg ew schwartz r weinstock h prevalence anogenital warts among participants private health plans united states 2003–2010 potential impact hum

In [ ]:
question = "Does the ACIP recommend one dose GARDASIL 9?"

In [ ]:
# Encode the inputs
# input_dict = tokenizer(question, context, return_tensors="pt")

In [ ]:
# n_docs = 5

In [1]:
# # Reshape input_ids_np to match the expected dimension
# input_ids_np = input_ids_np.reshape(1, -1)

# # Perform passage retrieval
# retrieved_doc_ids = retriever.retrieve(input_ids_np, n_docs=n_docs)

# # Get the retrieved documents
# retrieved_docs = retriever.batch_retrieve(retrieved_doc_ids)

In [ ]:
# import time
# t0 = time.time()

# question = "Is GARDASIL 9 recommended for Adults?"

# input_ids = tokenizer.question_encoder(question, return_tensors = "pt")["input_ids"]
# generated = model.generate(input_ids)

# generated_string = tokenizer.batch_decode(generated, skip_special_tokens = True)[0]
# print(generated_string)



---

With the RAG Generator. It can be seen that it doesn't perform well.

---



In [ ]:
import time

t0 = time.time()

question = "Does the ACIP recommend one dose GARDASIL 9?"

outputs = tokenizer.question_encoder(question, return_tensors="pt")
input_ids = outputs["input_ids"] 
attention_mask = outputs["attention_mask"] 

generated = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    num_return_sequences=8,
    max_length=50,
    temperature=0.8,
    num_beams = 5,
    min_length=30  # Set the desired minimum length here
)
generated_strings = tokenizer.batch_decode(generated, skip_special_tokens=True)
for i, generated_string in enumerate(generated_strings):
    print(f"Generated sequence {i + 1}: {generated_string}")

print(f"Time taken: {time.time() - t0:.2f} seconds")


Generated sequence 1:  6.5 u.s. pascals, m.c., m.n., m.c.,m.n.
Generated sequence 2:  6.5 u.s. pascals, m.c., m.n., m.c.,m.s.
Generated sequence 3:  6.5 u.s. pascals, m.c., m.n., m.c.,m.n.s.
Generated sequence 4:  6.5 u.s. pascals, m.c., m.n., m.c.,m.s.s.
Generated sequence 5:  6.5 u.s. pascals, m.c., m.n., m.c.,m.s.a.a.a.a.a.a.u.u.u.
Generated sequence 6:  doesn't need to be re-september 9, 2017., 2017.iocco.i.u.a.i.n.i.i.u.u.u.u.u.u.
Generated sequence 7:  doesn't need to be re-september 9, 2017., 2017.iocco.i.u.a.i.n.i.u.i.u.u.u.u.u.
Generated sequence 8:  doesn't need to be re-september 9, 2017., 2017.iocco.i.u.a.i.n.i.i.u.u.u.u.u.m.
Time taken: 419.42 seconds




---

**T5 Model**

---



In [ ]:
t5_model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
t5_tokenizer

T5Tokenizer(name_or_path='t5-base', vocab_size=32100, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_44

In [ ]:
question

'Does the ACIP recommend one dose GARDASIL 9?'

In [ ]:
inputs = t5_tokenizer.prepare_seq2seq_batch(question, retrieved_passages, return_tensors="pt")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3732: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is depr

In [ ]:
outputs = t5_model.generate(inputs["input_ids"], attention_mask=inputs["attention_mask"])


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
answers = t5_tokenizer.batch_decode(outputs, skip_special_tokens=True)


In [ ]:
# Generate answers from each passage
answers = []
for passage in retrieved_passages:
    # Concatenate the question and passage
    input_text = f"question: {question} context: {passage}"

    # Encode the input text
    input_ids = t5_tokenizer.encode(input_text, return_tensors="pt")

    # Generate the answer
    output = t5_model.generate(input_ids, num_beams = 5, max_length=50)

    # Decode and store the answer
    answer = t5_tokenizer.decode(output[0], skip_special_tokens=True)
    answers.append(answer)

# Print the generated answers
for i, answer in enumerate(answers):
    print(f"Answer {i+1}: {answer}")


Answer 1: 165 kreimer ar gonzalez p katki ha et al efficacy bivalent hpv 1618 vaccine anal hpv 1618 infection among young women
Answer 2: 20131031428–35 209 hariri markowitz le dunne ef unger er population impact hpv vaccines
Answer 3: u se 9valent human papillomavirus hpv vaccine updated hpv vaccination recommendations
Answer 4: 2 us b urden disease collaborators state us health 19902010 burden diseases injuries risk factors jama 2013310591–608 3 lee pn f ry js systematic review
Answer 5: 26 years bivalent hpv vaccine cervarix produced glaxosmithkline rixensart belgium licensed use females aged 9 25 years vaccines
Answer 6: 47–67 30000new mexico 9224 55 49–62 93000delaware 5150 55 48–63 43000new jersey 13179 54 49–60 400000texas 10783 53 47–59
Answer 7: sandra adamson fryhofer md atlanta georgia
Answer 8: 6 43 14 34 20 36staff member 5 36 17 41 22 40community member3 21 10 24 13 24sexwomen 11 79 31 76 42
Answer 9: june 4 june 5 started amoxicillin hospitalized died june 7three alzheim



---

**1. Concatenating Input Text:** The code combines the input question and the retrieved passages into a single string called `input_text`. The question is prefixed with "question:" and the passages are prefixed with "context:". This concatenation is done to provide the model with the question and relevant context for generating answers.

**2. Encoding Input Text:** The code tokenizes and encodes the `input_text` using the T5 tokenizer (`t5_tokenizer`). The encoded input is returned as a PyTorch tensor (`input_ids`).

**3. Answer Generation:** The code passes the encoded input tensor to the T5 model (`t5_model`) for answer generation. It specifies the number of beams to use during generation (`num_beams`) and the maximum length of the generated answer (`max_length`).

**4. Decoding and Storing the Answer:** The code decodes the generated answer by removing special tokens and converting the output tensor (`output[0]`) into a readable text using the T5 tokenizer (`t5_tokenizer.decode`). The decoded answer is then appended to the `answers` list.


---



In [ ]:
# # Generate answers from each passage
answers = []
# for passage in retrieved_passages:
#     # Concatenate the question and passage
input_text = f"question: {question} context: {retrieved_passages}"

    # Encode the input text
input_ids = t5_tokenizer.encode(input_text, return_tensors="pt")

    # Generate the answer
output = t5_model.generate(input_ids, num_beams = 5, max_length=50)

    # Decode and store the answer
answer = t5_tokenizer.decode(output[0], skip_special_tokens=True)
answers.append(answer)

# Print the generated answers
for i, answer in enumerate(answers):
    print(f"Answer {i+1}: {answer}")

Token indices sequence length is longer than the specified maximum sequence length for this model (1843 > 512). Running this sequence through the model will result in indexing errors


Answer 1: new jersey licensed use females males aged 9 26 years bivalent hpv vaccine cervarix produced glaxosmithkline rixensart belgium licensed use females aged 9 26 years


In [ ]:
for i, answer in enumerate(answers):
    print(f"Answer {i+1}: {answer}")

Answer 1: new jersey licensed use females males aged 9 26 years bivalent hpv vaccine cervarix produced glaxosmithkline rixensart belgium licensed use females aged 9 26 years


# Is GARDASIL 9 recommended for Adults?

In [ ]:
import re
question = "Is GARDASIL 9 recommended for Adults?"

In [ ]:
input_ids = q_tokenizer.encode(question, return_tensors = "pt")
input_ids = input_ids.to(device)
outputs = q_encoder(input_ids)
q_embed = outputs['pooler_output']
q_embed = q_embed.cpu().numpy()
print("Query Embedding:", q_embed.shape)

Query Embedding: (1, 768)


In [ ]:
D, I = index.search(q_embed, k= 10)
print("Closest matching indices:", I)
print("Inner Products:", D)

Closest matching indices: [[ 72 326 126 449 273  99  93 455  36 121]]
Inner Products: [[20.876705 20.6635   20.365509 20.290716 18.066357 17.460443 15.822736
  15.710029 15.13734  12.05438 ]]


In [ ]:
import textwrap 
retrieved_passages = []
retrieved_indices = []
wrapper = textwrap.TextWrapper(width = 80)
for i in I[0]:
  print('Index:', i)
  title = chunked_corpus['title'][i]
  passage = chunked_corpus['text'][i]
  retrieved_passages.append(passage)
  retrieved_indices.append(i)
  print('Article Title:' , title, '\n')
  print('Passage:')
  print(wrapper.fill(passage))
  print('')

Index: 72
Article Title: Employment and Activity Limitations Among Adults with Chronic Obstructive Pulmonary Disease 

Passage:
ronic obstructive pulmonary disease — united states 2013anne g wheaton phd1
timothy j cunningham phd1 earl ford md1 janet b croft phd1 author affiliations
end textinside296 mycoplasma pneumoniae outbreak longterm care facility —
nebraska 2014300 u se 9valent human papillomavirus hpv vaccine updated hpv
vaccination recommendations advisory committee immunization practices305 upda
ted recommendations use typhoid vaccine — advisory committee immunization
practices united states 2015309 announcement310 quickstatscontinuing education
examination available httpwwwcdcgovmmwrcmecontedinfohtmlweekly † available httpw

Index: 326
Article Title: Recommendations of the Advisory Committee on Immunization Practices 

Passage:
betes mellitus autoimmune thyroiditis receipt hpv4 156bivalent hpv vaccine
hpv2hpv2 efficacyfemales aged 15–25 yearshpv2 efficacy cin2 evaluated two
r

In [ ]:
question

'Is GARDASIL 9 recommended for Adults?'

In [ ]:
# # Generate answers from each passage
answers = []
# for passage in retrieved_passages:
#     # Concatenate the question and passage
input_text = f"question: {question} context: {retrieved_passages}"

    # Encode the input text
input_ids = t5_tokenizer.encode(input_text, return_tensors="pt")

    # Generate the answer
output = t5_model.generate(input_ids, num_beams = 10, max_length=50)

    # Decode and store the answer
answer = t5_tokenizer.decode(output[0], skip_special_tokens=True)
answers.append(answer)

# Print the generated answers
for i, answer in enumerate(answers):
    print(f"Answer {i+1}: {answer}")

Answer 1: noninfectious quadrivalent 9valent hpv vac cines 4vhpv 9vhpv gardasil 9 merck co inc whitehouse station new jersey licensed f


In [ ]:
for i, answer in enumerate(answers):
    print(f"Answer {i+1}: {answer}")

Answer 1: noninfectious quadrivalent 9valent hpv vac cines 4vhpv 9vhpv gardasil 9 merck co inc whitehouse station new jersey licensed f


# What were the past 3 recommendation changes for GARDASIL 9?

In [ ]:
import re
question = "What were the past 3 recommendation changes for GARDASIL 9?"

In [ ]:
input_ids = q_tokenizer.encode(question, return_tensors = "pt")
input_ids = input_ids.to(device)
outputs = q_encoder(input_ids)
q_embed = outputs['pooler_output']
q_embed = q_embed.cpu().numpy()
print("Query Embedding:", q_embed.shape)

Query Embedding: (1, 768)


In [ ]:
D, I = index.search(q_embed, k= 10)
print("Closest matching indices:", I)
print("Inner Products:", D)

Closest matching indices: [[ 72 449  99  93 273 455 126  36 121 110]]
Inner Products: [[22.115742 20.434309 20.160736 18.640837 17.500168 17.128311 16.69266
  14.707298 14.502974 14.388355]]


In [ ]:
import textwrap 
retrieved_passages = []
retrieved_indices = []
wrapper = textwrap.TextWrapper(width = 80)
for i in I[0]:
  print('Index:', i)
  title = chunked_corpus['title'][i]
  passage = chunked_corpus['text'][i]
  retrieved_passages.append(passage)
  retrieved_indices.append(i)
  print('Article Title:' , title, '\n')
  print('Passage:')
  print(wrapper.fill(passage))
  print('')

Index: 72
Article Title: Employment and Activity Limitations Among Adults with Chronic Obstructive Pulmonary Disease 

Passage:
ronic obstructive pulmonary disease — united states 2013anne g wheaton phd1
timothy j cunningham phd1 earl ford md1 janet b croft phd1 author affiliations
end textinside296 mycoplasma pneumoniae outbreak longterm care facility —
nebraska 2014300 u se 9valent human papillomavirus hpv vaccine updated hpv
vaccination recommendations advisory committee immunization practices305 upda
ted recommendations use typhoid vaccine — advisory committee immunization
practices united states 2015309 announcement310 quickstatscontinuing education
examination available httpwwwcdcgovmmwrcmecontedinfohtmlweekly † available httpw

Index: 449
Article Title: Recommendations of the Advisory Committee on Immunization Practices 

Passage:
int j cancer 2013132198–207 207 markowitz le hariri lin c et al reduction human
papillomavirus hpv prevalence among young women following hpv vaccine


In [ ]:
question

'What were the past 3 recommendation changes for GARDASIL 9?'

In [ ]:
answers = []
# for passage in retrieved_passages:
#     # Concatenate the question and passage
input_text = f"question: {question} context: {retrieved_passages}"

    # Encode the input text
input_ids = t5_tokenizer.encode(input_text, return_tensors="pt")

    # Generate the answer
output = t5_model.generate(input_ids, num_beams = 10, max_length=50)

    # Decode and store the answer
answer = t5_tokenizer.decode(output[0], skip_special_tokens=True)
answers.append(answer)

# Print the generated answers
for i, answer in enumerate(answers):
    print(f"Answer {i+1}: {answer}")

Answer 1: use 2dose schedule girls boys initiate vaccination series ages 9 14 years


In [ ]:
# Generate answers from each passage
answers = []
for passage in retrieved_passages:
    # Concatenate the question and passage
    input_text = f"question: {question} context: {passage}"

    # Encode the input text
    input_ids = t5_tokenizer.encode(input_text, return_tensors="pt")

    # Generate the answer
    output = t5_model.generate(input_ids, num_beams = 5, max_length=50)

    # Decode and store the answer
    answer = t5_tokenizer.decode(output[0], skip_special_tokens=True)
    answers.append(answer)

# Print the generated answers
for i, answer in enumerate(answers):
    print(f"Answer {i+1}: {answer}")

Answer 1: use typhoid vaccine
Answer 2: 53679–82 210
Answer 3: lee pn f ry js systematic review
Answer 4: 10783 53 47–59 1040000connecticut 7609 53 47–60 163000washington 11065 53 48–58 301000new york 8805 52 47–59 856000massachuset
Answer 5: females males aged 9 26 years bivalent hpv vaccine cervarix produced glaxosmithkline rixensart belgium licensed use females aged 9 25 years vaccines
Answer 6: sandra adamson fryhofer md atlanta georgia american geriatrics society kenneth schmader md durham north carolina america’
Answer 7: 21 10 24 13 24sexwomen 11 79 31 76 42
Answer 8: new recommendations use 2dose schedule girls boys initiate vaccination series ages 9 14 years three doses remain recommended persons initiate vaccination series ages 15 26 years immunocompromised persons
Answer 9: strict droplet precautions
Answer 10: attending physician collect np op swabs


# When did the GARDASIL 9 recommendations change?

In [ ]:
import re
question = "When did the GARDASIL 9 recommendations change?"

In [ ]:
input_ids = q_tokenizer.encode(question, return_tensors = "pt")
input_ids = input_ids.to(device)
outputs = q_encoder(input_ids)
q_embed = outputs['pooler_output']
q_embed = q_embed.cpu().numpy()
print("Query Embedding:", q_embed.shape)

Query Embedding: (1, 768)


In [ ]:
D, I = index.search(q_embed, k= 10)
print("Closest matching indices:", I)
print("Inner Products:", D)

Closest matching indices: [[437 449 455  72 326 273  99  93 126  36]]
Inner Products: [[33.825714 24.246387 24.026375 23.92072  20.311127 20.272831 20.13876
  17.613972 15.34108  14.980211]]


In [ ]:
import textwrap 
retrieved_passages = []
retrieved_indices = []
wrapper = textwrap.TextWrapper(width = 80)
for i in I[0]:
  print('Index:', i)
  title = chunked_corpus['title'][i]
  passage = chunked_corpus['text'][i]
  retrieved_passages.append(passage)
  retrieved_indices.append(i)
  print('Article Title:' , title, '\n')
  print('Passage:')
  print(wrapper.fill(passage))
  print('')

Index: 437
Article Title: Recommendations of the Advisory Committee on Immunization Practices 

Passage:
skinner sr et al efficacy human papillomavirus hpv1618 as04adjuvanted vaccine
women aged 15–25 years without serological evidence previous exposure hpv1618
int j cancer 2012131106–16 164 herrero r quint w hildesheim et al reduced
prevalence oral human papillomavirus hpv 4 years bivalent hpv vaccination
randomized clinical trial costa rica plos one 20138e68329 165 kreimer ar
gonzalez p katki ha et al efficacy bivalent hpv 1618 vaccine anal hpv 1618
infection among young women nested analysis within costa rica vaccine rial
lancet oncol 201112862–70 166 naud ps rotelimartins cm de carval

Index: 449
Article Title: Recommendations of the Advisory Committee on Immunization Practices 

Passage:
int j cancer 2013132198–207 207 markowitz le hariri lin c et al reduction human
papillomavirus hpv prevalence among young women following hpv vaccine
introduction united states national health nutr

In [ ]:
answers = []
# for passage in retrieved_passages:
#     # Concatenate the question and passage
input_text = f"question: {question} context: {retrieved_passages}"

    # Encode the input text
input_ids = t5_tokenizer.encode(input_text, return_tensors="pt")

    # Generate the answer
output = t5_model.generate(input_ids, num_beams = 10, max_length=50)

    # Decode and store the answer
answer = t5_tokenizer.decode(output[0], skip_special_tokens=True)
answers.append(answer)

# Print the generated answers
for i, answer in enumerate(answers):
    print(f"Answer {i+1}: {answer}")

Answer 1: 2015309 announcement310 quickstatscontinuing education examination available httpwwwcdcgovmmwrcmecontedinfohtmlweekly  available httpw'


In [ ]:
# Generate answers from each passage
answers = []
for passage in retrieved_passages:
    # Concatenate the question and passage
    input_text = f"question: {question} context: {passage}"

    # Encode the input text
    input_ids = t5_tokenizer.encode(input_text, return_tensors="pt")

    # Generate the answer
    output = t5_model.generate(input_ids, num_beams = 5, max_length=50)

    # Decode and store the answer
    answer = t5_tokenizer.decode(output[0], skip_special_tokens=True)
    answers.append(answer)

# Print the generated answers
for i, answer in enumerate(answers):
    print(f"Answer {i+1}: {answer}")

Answer 1: 2012131106–16 164 herrero r quint w hildesheim et al reduced prevalence oral human papillomavirus hpv 4 years bivalent hp
Answer 2: 20131031428–35 209 hariri markowitz le dunne ef unger er population impact hpv vaccines
Answer 3: david kimberlin md birmingham alabama
Answer 4: 2015309 announcement310 quickstatscontinuing education examination available httpwwwcdcgovmmwrcmecontedinfohtmlweekly
Answer 5: 949 95 ci 877–984 table 4 160 statistically significant efficacy demonstrated individually hpv 16 hpv 18related lesions itt
Answer 6: 9 25 years vaccines composed typespecific hpv l1 protein major capsid protein hpv expression l1 protein using recombinant dna technology produces vlps vaccines
Answer 7: 2013 621–96 2 us b urden disease collaborators state us health 19902010 burden diseases risk factors jama 2013310591–608 3 lee pn f ry js systematic review
Answer 8: 47–67 30000new mexico 9224 55 49–62 93000delaware 5150 55 48–63 43000new jersey 13179 54 49–60 400000texas 10783 5

In [ ]:
from transformer import BartTokenizer, BartForConditionalGene